<div style="margin-left: auto;
            margin-right: auto;
            width: 80%">

| | Amager Vest | Amager Øst | Bispebjerg | Brønshøj-Husum | Indre By | Nørrebro | Valby | Vanløse | Vesterbro - Kongens Enghave | Østerbro | 
|--------|-----------|------------|-----------|----------|-----------|----------|--------|--------|--------|--------|
|  Offentlige  |   221  | 125      | 45    | 53   | 864    | 372   |91   |25   |456   | 506 |
|  Private     |   119  | 48     | 150   | 68   | 518    | 418   |111   |75   |438  | 818 |
|  Ikke registreret  |  32  | 76  | 0  | 0   | 296    | 4   |1   |0   |23   | 83 |



_Table X: Amount of public, private and not registred bike racks._
</div>

Amager Vest - ca. 59,4\% offentlige\
Amager Øst - ca. 50,2\% offentlige\
Bispebjerg - ca. 23,1\% offentlige\
Brønshøj-Husum - ca. 43,8\% offentlige\
Indre By - ca. 51,5\% offentlige\
Nørrebro - ca. 46,9\% offentlige\
Valby - ca. 44,8\% offentlige\
Vanløse - 25\% offentlige\
Vesterbro - Kongens Enghave - ca. 49,7\% offentlige\
Østerbro - ca. 36,0\% offentlige

### Part III

Starter med at holde fokus på Amager Vest

In [1]:
import pandas as pd
from bokeh.transform import jitter
from bokeh.palettes import Category20
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import ColumnDataSource, HoverTool, DatetimeTicker, Range1d, LinearAxis, Legend


#ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
## Load data
file = "..\..\Data\cykelstativ.csv"
df = pd.read_csv(file)

df = df[df["antal_pladser"] != 0]

In [3]:
file = "..\..\Data\Befolkning.csv"
df_befolkning = pd.read_csv(file,sep=',',encoding='latin-1', header=1)

#delete last row of df_befolkning
df_befolkning = df_befolkning[:-1]

#sort by first collumn
df_befolkning = df_befolkning.sort_values(by=' ')


In [4]:
## Select district
bydele = ['Amager Vest', 'Amager Øst', 'Bispebjerg', 'Brønshøj-Husum', 'Indre By',
       'Nørrebro', 'Valby', 'Vanløse', 'Vesterbro-Kongens Enghave',
       'Østerbro']

bydel = "Indre By"
pop_bydel = f"Bydel - {bydel}"

In [5]:
## Bike racks / spots in Amager Vest
df_3 = df[df['bydel'] == bydel]
print(f"Total amount of spots: {df_3['antal_pladser'].sum()}")

## Der er en del steder hvor datoen er NaN, men der er angivet et antal pladser (2066 rækker)

nan_count = df_3['reg_dato'].isna().sum()
extra_spots = df_3[df_3['reg_dato'].isna()]['antal_pladser'].sum()
print(f"Total amount of spots without date: {extra_spots}")

## Frasorterer NaN værdier
dato_3 = df_3.groupby('reg_dato')['antal_pladser'].sum()
dato_3.index = dato_3.index.str[:10]
print(f"Total amount of spots with date: {dato_3.sum()}")


## Make the accumulated sum
dato_3.values[0] += extra_spots #NOTE extra spots as start?
accum_sum = dato_3.cumsum()


Total amount of spots: 26731.0
Total amount of spots without date: 4383.0
Total amount of spots with date: 22348.0


In [6]:
## Population in Amager Vest
df_befolkning_3 = df_befolkning


## Changing quntiles to months
mapping = {'K1': '-01', 'K2': '-04', 'K3': '-07', 'K4': '-10'}

# Rename columns
df_befolkning_3 = df_befolkning_3.rename(columns=lambda x: x[:4] + mapping.get(x[4:], ''))
df_befolkning_3.iloc[8, 0] = 'Bydel - Vesterbro-Kongens Enghave'
df_befolkning_3

,,2000-01,2001-01,2002-01,2002-04,2002-07,2002-10,2003-01,2003-04,2003-07,...,2021-10,2022-01,2022-04,2022-07,2022-10,2023-01,2023-04,2023-07,2023-10,2024-01
9,Bydel - Amager Vest,45397.0,45608.0,45695.0,45647.0,45523.0,45673.0,45631.0,45630.0,45409.0,...,83966.0,84561.0,85520.0,85505.0,86352.0,86668.0,87669.0,87226.0,88306.0,88380.0
8,Bydel - Amager Øst,47533.0,48196.0,48428.0,48449.0,48358.0,48357.0,48443.0,48492.0,48330.0,...,61455.0,61193.0,61411.0,61663.0,62101.0,62197.0,62718.0,62476.0,63068.0,62818.0
7,Bydel - Bispebjerg,44251.0,44641.0,45163.0,45217.0,45024.0,45141.0,44997.0,45084.0,44979.0,...,53934.0,53845.0,53957.0,54017.0,54239.0,54346.0,54758.0,54626.0,55044.0,54980.0
6,Bydel - Brønshøj-Husum,39657.0,39854.0,40008.0,39984.0,39923.0,40015.0,39989.0,40015.0,40060.0,...,44053.0,44005.0,43838.0,43803.0,43928.0,43933.0,43831.0,43690.0,44457.0,44467.0
0,Bydel - Indre By,43691.0,43978.0,44125.0,44168.0,44112.0,44307.0,44460.0,44512.0,44616.0,...,56413.0,56233.0,56601.0,56338.0,56813.0,56446.0,56838.0,56395.0,57153.0,56668.0
2,Bydel - Nørrebro,72993.0,73502.0,73865.0,73962.0,73813.0,74049.0,73836.0,73957.0,73529.0,...,78591.0,78588.0,78552.0,78402.0,78508.0,78583.0,78779.0,78962.0,79318.0,79067.0
4,Bydel - Valby,46209.0,46189.0,46271.0,46348.0,46264.0,46273.0,46211.0,46225.0,46145.0,...,63496.0,64144.0,64518.0,65176.0,65889.0,66435.0,66404.0,65877.0,65933.0,65780.0
5,Bydel - Vanløse,35822.0,36049.0,36233.0,36125.0,36069.0,36222.0,36228.0,36193.0,36146.0,...,40805.0,40781.0,40709.0,40598.0,40657.0,40633.0,40757.0,40710.0,40776.0,40709.0
3,Bydel - Vesterbro-Kongens Enghave,50312.0,50756.0,51037.0,51041.0,50878.0,50981.0,51035.0,51179.0,50921.0,...,77074.0,77283.0,77442.0,77704.0,78894.0,79646.0,80002.0,80363.0,81143.0,80920.0
1,Bydel - Østerbro,66673.0,66877.0,67017.0,67248.0,67095.0,67495.0,67554.0,67700.0,67630.0,...,79900.0,79952.0,80345.0,80356.0,80655.0,80605.0,80842.0,80779.0,81326.0,81278.0


In [7]:
accum_sum = pd.DataFrame({'Date': accum_sum.index, 'Spots': accum_sum.values})
accum_sum['Date'] = pd.to_datetime(accum_sum['Date'])

In [9]:
## Normal - NOTE colors + noget galt når man holder musen over

# Create a Bokeh plot
p = figure(title='Bike Spots vs. Population ', x_axis_label='Year', y_axis_label='Population',
           width=1200, height=600, #width=900, height=450,
           x_axis_type="datetime")
p.extra_y_ranges = {"y2": Range1d(start = 0, end = 27000)}
p.add_layout(LinearAxis(y_range_name = "y2", axis_label = 'Bike Spots'), 'right')



items = []
items2 = []
b = {}
b2 = {}

# Add hover tool

for d in range(len(bydele)):

    ##----
    df_3 = df[df['bydel'] == bydele[d]]    

    extra_spots = df_3[df_3['reg_dato'].isna()]['antal_pladser'].sum()

    dato_3 = df_3.groupby('reg_dato')['antal_pladser'].sum()
    dato_3.index = dato_3.index.str[:10]

    ## Make the accumulated sum
    dato_3.values[0] += extra_spots #NOTE extra spots as start?
    accum_sum = dato_3.cumsum()

    df_temp = pd.DataFrame({'Date': pd.to_datetime(accum_sum.index), 'Spots': accum_sum.values})
    df_temp['FormattedDate'] = df_temp['Date'].dt.strftime('%Y-%m-%d')  # Format to readable date
    df_temp['district'] = bydele[d] + ' - Spots'
    Data = df_temp[['Date', 'Spots', 'FormattedDate', 'district']]

    source = ColumnDataSource(Data)

    b[d] = p.line(x='Date', y=jitter('Spots', width=0.6, range=p.y_range),  source=source, muted_alpha=0, muted=True, y_range_name="y2", color=Category20[20][d])
    # Create legend items
    items.append((bydele[d] + ' - Spots', [b[d]]))

    hover = HoverTool(renderers=[b[d]], tooltips=[('District', '@district'), ('Year', '@FormattedDate'), ('Value', '@Spots')])
    p.add_tools(hover)

    ##----
    if df_befolkning_3.iloc[d][0] == f'Bydel - {bydele[d]}': 
        df_temp_bef = df_befolkning_3.iloc[d]

        df_temp2 = pd.DataFrame({'Date': pd.to_datetime(df_temp_bef.index[1:]), 'Population': df_temp_bef.values[1:]})
        df_temp2['FormattedDate'] = df_temp2['Date'].dt.strftime('%Y-%m-%d')  # Format to readable date
        df_temp2['district'] = df_befolkning_3.iloc[d][0][8:]  + ' - Population'
        Data2 = df_temp2[['Date', 'Population', 'FormattedDate', 'district']]
        
        source2 = ColumnDataSource(Data2)

        b2[d] = p.line(x='Date', y=jitter('Population', width=0.6, range=p.y_range, ),  source=source2, muted_alpha=0, color=Category20[20][d+10], muted=False)
        items.append((bydele[d] + ' - Population', [b2[d]])) # Create legend items

        # Add hover tool
        hover = HoverTool(renderers=[b2[d]], tooltips=[('District', '@district'), ('Year', '@FormattedDate'), ('Value', '@Population')])
        p.add_tools(hover)
    


# Create legend
legend = Legend(items=items, location='center_right')
p.add_layout(legend, 'right')

p.legend.glyph_height = 10
p.legend.label_height = 10

p.xaxis.ticker = DatetimeTicker(desired_num_ticks=10)  # Increase or decrease as needed


p.x_range.range_padding = 0
p.ygrid.grid_line_color = None
p.legend.click_policy = "mute"

# Show the plot
show(p)

# output_file("bokeh_population_spots.html")
# save(p)

'c:\\Users\\julie\\OneDrive\\Skrivebord\\Social Data Analysis, 02806\\Knorsker.github.io\\A3\\bokeh_population_spots.html'

In the interactive plot, the first noticeable feature is the population progress across various districts in Copenhagen. The population data spans from the year 2000 to 2024, with measurements taken quarterly from 2002 to 2024, and once each in 2000 and 2001.
Additionally, is the plot showing the visualization of the cumulative number of bike spots across different districts over the years. The initial value for this cumulative sum is the total number of bike spots without registration dates.

A closer analysis of the plot reveals that while the populations of most districts remain relatively stable over the years, notable increases stand out in _Amager Vest_, _Vesterbro - Kongens Enghave_ and _Valby_. Specifically, these districts have experienced population increases of $42,983$, $30,608$, and $19,571$ respectively.

Focusing on Amager Vest, the corresponding trend in bike spots over time can be plotted. A noticeable increase is observed in bike spots, especially from July 2014 to February 2017. Interestingly, this increasing in bike infrastructure does not directly correlate with the population growth in Amager Vest. \
When examining the data for Vesterbro - Kongens Enghave, the number of bike spots remains relatively stable from 2001 to 2016. However, a significant change occurs in 2017, indicating proactive efforts by the municipality to accommodate the population growth.\
For the final district, Valby, which experienced an increase over the 24 years, an analysis of the trends in bike spots and population can reveal a parallel growth in population and the cumulative number of bike spots. Despite the increase in population, there is no corresponding increase in bike spots.


When plotting the cumulative sums of bike spots across different districts in Copenhagen, a notable increase is observed particularly in the years 2016 and 2017. Specifically, districts such as Indre By saw a significant rise of approximately 12,000 spots, followed by Vesterbro - Kongens Enghave with an increase of about 4,000 spots, and Østerbro with a rise of 2,000 spots. Conversely, districts like Vanløse, Brønshøj - Husum, and Bispebjerg exhibited minimal growth, remaining among the districts with the fewest bike spots.\
Another noteworthy observation is the upward trend that emerged around the year 2001, during which many districts experienced growth as well. This could potentially be attributed to the initiation of the project and the implementation of bike rack and spot counting measures.




Alle bike spots: 
- Significant change around 2017
- Stigning lige i starten, hvor projektet er startet. 

Indre by:
- Stor stigning i bike spots, men ikke population - ikke så mange der bor i indre by, men flere og flere der arbejder


Diskussion:
- Tager ikke højde for cykelskure der bliver nedlagt.

In [ ]:
print(f'Amager Vest: 2000 = {df_befolkning_3.iloc()[0,1]}, 2024 = {df_befolkning_3.iloc()[0,-1]}')
print(f'Stigning: {df_befolkning_3.iloc()[0,-1] - df_befolkning_3.iloc()[0,1]}')
print(' ')
print(f'Valby: 2000 = {df_befolkning_3.iloc()[6,1]}, 2024 = {df_befolkning_3.iloc()[6,-1]}')
print(f'Stigning: {df_befolkning_3.iloc()[6,-1] - df_befolkning_3.iloc()[6,1]}')
print(' ')
print(f'Vesterbro - Kongens Enghave Vest: 2000 = {df_befolkning_3.iloc()[8,1]}, 2024 = {df_befolkning_3.iloc()[8,-1]}')
print(f'Stigning: {df_befolkning_3.iloc()[8,-1] - df_befolkning_3.iloc()[8,1]}')
print(' ')


Amager Vest: 2000 = 45397.0, 2024 = 88380.0
Stigning: 42983.0
 
Valby: 2000 = 46209.0, 2024 = 65780.0
Stigning: 19571.0
 
Vesterbro - Kongens Enghave Vest: 2000 = 50312.0, 2024 = 80920.0
Stigning: 30608.0
 
